In [18]:
import cv2
import numpy as np
import os
import glob

In [19]:
def add_random_moire(image, amplitude=30, frequency=0.08, num_patches=3):

    rows, cols = image.shape[:2]
    img_float = image.astype(np.float32)

    x = np.arange(cols)
    y = np.arange(rows)
    X, Y = np.meshgrid(x, y)


    r_wave = amplitude * np.sin(2 * np.pi * frequency * X + 0)
    g_wave = amplitude * np.sin(2 * np.pi * frequency * X + 2)
    b_wave = amplitude * np.sin(2 * np.pi * frequency * X + 4)

    moire_noise = cv2.merge([b_wave, g_wave, r_wave])
    full_moire_img = np.clip(img_float + moire_noise, 0, 255).astype(np.uint8)

    selection_mask = np.zeros((rows, cols), dtype=np.float32)
    for _ in range(num_patches):
        size = np.random.randint(min(rows, cols) // 6, min(rows, cols) // 3)
        cx = np.random.randint(0, cols)
        cy = np.random.randint(0, rows)
        cv2.circle(selection_mask, (cx, cy), size, 1.0, -1)

    selection_mask = cv2.GaussianBlur(selection_mask, (31, 31), 0)
    selection_mask_3ch = cv2.merge([selection_mask, selection_mask, selection_mask])

    final_image = (img_float * (1 - selection_mask_3ch) +
                   full_moire_img.astype(np.float32) * selection_mask_3ch)
    final_image = np.clip(final_image, 0, 255).astype(np.uint8)

    diff = cv2.absdiff(image, final_image)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(diff_gray, 15, 255, cv2.THRESH_BINARY)

    return final_image, mask

In [20]:
def process_folder(input_folder_path, output_base_path):

    images_output_dir = os.path.join(output_base_path, "moire_images")
    masks_output_dir = os.path.join(output_base_path, "moire_masks")

    os.makedirs(images_output_dir, exist_ok=True)
    os.makedirs(masks_output_dir, exist_ok=True)

    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(input_folder_path, ext)))

    if not image_files:
        print(f"Error: No images found in {input_folder_path}")
        return

    print(f"Found {len(image_files)} images. Starting Moire processing...")

    for file_path in image_files:
        img = cv2.imread(file_path)
        if img is None:
            print(f"Skipping unreadable file: {file_path}")
            continue

        filename = os.path.basename(file_path)

        try:
            defective_img, mask = add_random_moire(img)

            cv2.imwrite(os.path.join(images_output_dir, filename), defective_img)
            cv2.imwrite(os.path.join(masks_output_dir, filename), mask)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    print(f"Done! Images saved in: {images_output_dir}")
    print(f"Masks saved in: {masks_output_dir}")

In [21]:
print("---------- start of Add random moire ----------")

SOURCE_DIR ="D:\dataset\Samsung\Original photos"
OUTPUT_BASE_DIR = "D:\dataset\Samsung\Add random moire"
process_folder(SOURCE_DIR, OUTPUT_BASE_DIR)

print("---------- end of Add random moire ----------")


---------- start of Add random moire ----------
Found 2745 images. Starting Moire processing...
Done! Images saved in: D:\dataset\Samsung\Add random moire\moire_images
Masks saved in: D:\dataset\Samsung\Add random moire\moire_masks
---------- end of Add random moire ----------
